In [1]:
# first of all, let's check that avro tools work

!java -jar /usr/bin/avro.jar

Version 1.12.0
 of Apache Avro
Copyright 2010-2015 The Apache Software Foundation

This product includes software developed at
The Apache Software Foundation (https://www.apache.org/).
----------------
Available tools:
    canonical  Converts an Avro Schema to its canonical form
          cat  Extracts samples from files
      compile  Generates Java code for the given schema.
       concat  Concatenates avro files without re-compressing.
        count  Counts the records in avro files or folders
  fingerprint  Returns the fingerprint for the schemas.
   fragtojson  Renders a binary-encoded Avro datum as JSON.
     fromjson  Reads JSON records and writes an Avro data file.
     fromtext  Imports a text file into an avro data file.
      getmeta  Prints out the metadata of an Avro data file.
    getschema  Prints out schema of an Avro data file.
          idl  Generates a JSON schema or protocol from an Avro IDL file
 idl2schemata  Extract JSON schemata of the types from an Avro IDL fil

In [4]:
# check that Spark client works

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
        .config("spark.sql.extensions", "org.projectnessie.spark.extensions.NessieSparkSessionExtensions, org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
        #.config("spark.sql.warehouse.dir", "s3://iceberg")
        .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog")    
        .config("spark.sql.catalog.iceberg.warehouse", "file:///opt/warehouse/iceberg")
        #.config("spark.sql.catalog.iceberg.type", "nessie")
        .config("spark.sql.catalog.iceberg.type", "rest")
        #.config("spark.sql.catalog.iceberg.uri", "http://localhost:19120/api/v2")
        .config("spark.sql.catalog.iceberg.uri", "http://nessie:19120/iceberg/main/")
        #.config("spark.sql.catalog.iceberg.ref", "main")
        .config("spark.sql.catalog.iceberg.cache-enabled", False)
        .getOrCreate()
)
spark.sparkContext.setLogLevel('ERROR')

spark.sql("""
    SHOW TABLES FROM iceberg.default
""").show()

Py4JJavaError: An error occurred while calling o37.sql.
: org.projectnessie.client.http.HttpClientException: Failed to execute GET request against 'http://localhost:19120/api/v2/trees/main?fetch=MINIMAL'.
	at org.projectnessie.client.http.impl.BaseHttpRequest.processResponse(BaseHttpRequest.java:126)
	at org.projectnessie.client.http.impl.BaseHttpRequest.executeRequest(BaseHttpRequest.java:69)
	at org.projectnessie.client.http.HttpRequest.get(HttpRequest.java:112)
	at org.projectnessie.client.http.HttpRequestWrapper.unwrap(HttpRequestWrapper.java:72)
	at org.projectnessie.client.http.HttpRequestWrapper.get(HttpRequestWrapper.java:52)
	at org.projectnessie.client.rest.v2.HttpGetReference.get(HttpGetReference.java:41)
	at org.apache.iceberg.nessie.NessieIcebergClient.loadReference(NessieIcebergClient.java:127)
	at org.apache.iceberg.nessie.NessieIcebergClient.lambda$new$0(NessieIcebergClient.java:94)
	at org.apache.iceberg.relocated.com.google.common.base.Suppliers$NonSerializableMemoizi

In [ ]:
# check that Trino client works

from trino.dbapi import connect

trino_connection = connect(
    host="trino",
    port=8080,
    user="iceberg",
)
trino = trino_connection.cursor()
trino.execute("SELECT * FROM system.runtime.nodes")

rows = trino.fetchall()
print(rows)

In [ ]:
# check that Python client works

from pyiceberg.catalog.hive import HiveCatalog
from pyiceberg.exceptions import NoSuchTableError

catalog = HiveCatalog(name="iceberg", uri="thrift://hivemetastore:9083")

try:
    catalog.load_table("default.sample")
except NoSuchTableError as e:
    print(e)